### Factorization Machines:
### This notebook illustrates a pure python implementation of Steffen Rendle's Factorization Machines.
### Note that there are other good libraries out there that implement Factorization Machines: Rendle's libfm in C++ (http://www.libfm.org/) and pyFM (https://github.com/coreylynch/pyFM), which is a cythonized python implementation. 

### The purpose of implementing Factiorzation Machines here is three fold: 1) to understand the algorithm (there is no better way than implementing yourself!), 2) to provide a pure python implementation that is easy to experiment with, and 3) to scale this out for large data sets on MPP databases such as GPDB and HAWQ via PL/Python UDFs.

### Reference: Factorization Machines By Steffen Rendle
### http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf


#### Import the Necessary Python Packages

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas
from matplotlib import pyplot as plt
import seaborn as sns

In [8]:
class FactorizationMachines:
    def __init__(
            self,
            lambda0=1,
            lambda1=1,
            lambda2=1,
            n_iter=100,
            learning_rate=1,
            k=50,
            random_seed=None
    ):
        self.lambda0 = lambda0
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.k = k
        self.random_seed = random_seed
        if (self.random_seed is not None):
            self.random_seed = int(self.random_seed)
    
    def __updateParametersMiniBatch(self, X,y,B):
        y_hat = self.__predict(X)
        y_hat = np.minimum(self.maxY, y_hat)
        y_hat = np.maximum(self.minY, y_hat)
        term1 = 2*(y_hat - y[:,np.newaxis])
        gradW0 = np.sum(term1, axis = 0) + self.lambda0*self.w0
        if(gradW0.shape[0] <> 1 or gradW0.shape[1] <> 1):
            raise ValueError('Gradient W0 has more than 1 element')
        gradW = np.sum(np.multiply(term1,X), axis = 0) + self.lambda1*self.w
        if(gradW.shape[1] <> X.shape[1]):
            raise ValueError('Gradient W has incorrect number of elements')   
        # Alternate implementation instead of the for loop over instances.
        #gradV = np.multiply(term1,X).T*(X*self.V)
        #term2 = np.array(map(lambda row: np.multiply(self.V,row.T), np.multiply(term1,np.square(X))))
        #term2 = np.sum(term2,axis=0)
        #gradV -= term2
        gradV = np.zeros(self.V.shape)
        D = X.shape[0]
        instances = range(0,D)
        for d in instances:
            xd = X[d,:]
            xdV = xd*self.V
            gradV += term1[d][0,0] * (
                        np.multiply(xd.T,np.repeat(xdV,xd.shape[1],axis=0)) -
                        np.multiply(self.V,np.square(xd.T))
                    ) 
        
        gradV = gradV + self.lambda2*self.V
        self.w0 -= (self.learning_rate/B*1.0)*gradW0;
        self.w  -= (self.learning_rate/B*1.0)*gradW;
        self.V  -= (self.learning_rate/B*1.0)*gradV
        
    def __updateParameters(self, X,y):
        y_hat = self.__predict(X)
        y_hat = np.minimum(self.maxY, y_hat)
        y_hat = np.maximum(self.minY, y_hat)
        term1 = 2*(y_hat - y)
        gradW0 = term1 + self.lambda0*self.w0
        if(gradW0.shape[0] <> 1 or gradW0.shape[1] <> 1):
            raise ValueError('Gradient W0 has more than 1 element')
        gradW = np.multiply(term1,X) + self.lambda1*self.w
        if(gradW.shape[1] <> X.shape[1]):
            raise ValueError('Gradient W has incorrect number of elements')   
        gradV = np.zeros(self.V.shape)
        xdV = X*self.V
        gradV += term1[0,0] * (
                    np.multiply(X.T,np.repeat(xdV,X.shape[1],axis=0)) -
                    np.multiply(self.V,np.square(X.T))
                ) 
        gradV += self.lambda2*self.V
        self.w0 -= self.learning_rate*gradW0;
        self.w  -= self.learning_rate*gradW;
        self.V  -= self.learning_rate*gradV
    
    def __predict(self,X):
        return (self.w0 + 
                X*self.w.T + 
                0.5*(np.sum(np.square(X*self.V) - (np.square(X)*np.square(self.V)),axis=1))
               )
    
    def fit(self,X,y):
        if type(X) != np.matrix:
            X = np.matrix(X)
        if type(y) != np.array:
            y = np.array(y)
        if (len(X.shape)!= 2):
            raise ValueError('X should be a 2-D matrix')
        if (X.shape[0] != len(y)):
            raise ValueError('X and y should contain the same number of examples')
        D = X.shape[0]
        n = X.shape[1]
        self.minY = min(y)
        self.maxY = max(y)
        self.w0 = 0
        self.w = np.zeros([1,n])
        self.V = np.random.normal(scale=0.1,size=(n, self.k))
        indx = range(0,D)
        minibatch_size = 500
        if (self.random_seed is not None):
            np.random.seed(self.random_seed)
        if (D >= 1000):
            print("Minibatch size: " + str(minibatch_size))
            for i in range(0,self.n_iter):
                print("Epoch number: " + str(i))
                shuffledIndx = np.random.permutation(indx)
                start = 0;
                end = minibatch_size
                while (start < D):
                    xbatch = X[shuffledIndx[start:end],:]
                    ybatch = y[shuffledIndx[start:end]]
                    self.__updateParametersMiniBatch(xbatch,ybatch,xbatch.shape[0])
                    start += minibatch_size
                    end += minibatch_size
                    if (end > D):
                        end = D
        else:
            for i in range(0, self.n_iter):
                print("Epoch number: " + str(i))
                shuffledIndx = np.random.permutation(indx)
                for d in shuffledIndx:
                    xd = X[d,:]
                    yd = y[d]
                    self.__updateParameters(xd,yd)

    def predict(self,X,y=None):
        if type(X) != np.matrix:
            X = np.matrix(X)
        if type(y) != np.array:
            y = np.array(y)
        y_hat = self.__predict(X)
        y_hat = np.minimum(self.maxY, y_hat)
        y_hat = np.maximum(self.minY, y_hat)
        return y_hat

### Experiment with MovieLens 100-K Ratings Data
### The data loading code below is copied from https://github.com/coreylynch/pyFM, a fast, cythonized python library for Fatorization Machines

In [4]:
from sklearn.feature_extraction import DictVectorizer

# Read in data
def loadData(filename,path="/Users/gmuralidhar/Projects/MLDatasets/movielens-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split('\t')
            data.append({ "user_id": str(user), "movie_id": str(movieid)})
            y.append(float(rating))
            users.add(user)
            items.add(movieid)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")
v = DictVectorizer()
X_train = v.fit_transform(train_data).toarray()
X_test = v.transform(test_data).toarray()
print X_train.shape
print X_test.shape
print len(train_users)
print len(train_items)

(90570, 2623)
(9430, 2623)
943
1680


In [9]:
f = FactorizationMachines(lambda0 = 0.01, lambda1 = 0.01, lambda2 = 0.01, k = 10, n_iter = 100, learning_rate = 0.5)
f.fit(X_train,y_train)

Minibatch size: 500
Epoch number: 0
Epoch number: 1
Epoch number: 2
Epoch number: 3
Epoch number: 4
Epoch number: 5
Epoch number: 6
Epoch number: 7
Epoch number: 8
Epoch number: 9
Epoch number: 10
Epoch number: 11
Epoch number: 12
Epoch number: 13
Epoch number: 14
Epoch number: 15
Epoch number: 16
Epoch number: 17
Epoch number: 18
Epoch number: 19
Epoch number: 20
Epoch number: 21
Epoch number: 22
Epoch number: 23
Epoch number: 24
Epoch number: 25
Epoch number: 26
Epoch number: 27
Epoch number: 28
Epoch number: 29
Epoch number: 30
Epoch number: 31
Epoch number: 32
Epoch number: 33
Epoch number: 34
Epoch number: 35
Epoch number: 36
Epoch number: 37
Epoch number: 38
Epoch number: 39
Epoch number: 40
Epoch number: 41
Epoch number: 42
Epoch number: 43
Epoch number: 44
Epoch number: 45
Epoch number: 46
Epoch number: 47
Epoch number: 48
Epoch number: 49
Epoch number: 50
Epoch number: 51
Epoch number: 52
Epoch number: 53
Epoch number: 54
Epoch number: 55
Epoch number: 56
Epoch number: 57
Epoc

In [10]:
print("-------------------------------------------------- w0 -------------------------------------------------") 
print ("\n")
print f.w0
print ("\n")
print("-------------------------------------------------- w --------------------------------------------------") 
print ("\n")
print f.w
print ("\n")
print("-------------------------------------------------- V --------------------------------------------------") 
print ("\n")
print f.V
print ("\n")
print np.reshape(f.V,(1,26230))

-------------------------------------------------- w0 -------------------------------------------------


[[ 3.34656446]]


-------------------------------------------------- w --------------------------------------------------


[[ 0.61328145  0.47469686  0.90242212 ...,  0.02495267  0.06471322
   0.02054506]]


-------------------------------------------------- V --------------------------------------------------


[[-0.40870045  0.03017888 -0.20970543 ..., -0.47110778  0.57152537
   0.05472403]
 [-0.01690139  0.47304651  0.0616384  ...,  0.24899617 -0.01998918
   0.38415891]
 [ 0.00772684 -0.31082075  0.38315543 ...,  0.72053736  0.26878315
   0.38685551]
 ..., 
 [-0.73041426 -0.43490667  0.33568821 ...,  0.4852243  -0.11128964
  -0.91447259]
 [-0.13421457 -0.35637099 -0.256242   ...,  0.03540553  0.0512204
   0.45753731]
 [-0.62530596  0.43564263  0.02631099 ...,  0.21032114 -0.05804412
   0.24561888]]


[[-0.40870045  0.03017888 -0.20970543 ...,  0.21032114 -0.05804412
   0.245618

In [11]:
print ("Number of test examples = " + str(X_test.shape[0]))
y_hat = f.predict(np.matrix(X_test))
print ("Number of predicted examples = " + str(y_hat.shape[0]))
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_hat)
print ("Mean squared error = " + str(mse))

Number of test examples = 9430
Number of predicted examples = 9430
Mean squared error = 0.922828594099
